## Student Group Partition

In [1]:
import pandas as pd
import numpy as np
from rsome import ro
from sklearn.preprocessing import LabelEncoder
from rsome import grb_solver as grb

In [2]:
# att_list = ["No.", "Name", "Prog", "Year", "Gender", "Student Type", "Nationality", "VMS Acc"]
att_select = ["No.", "Name", "Prog", "Year", "Gender", "Table"]
profile = pd.read_csv("Student_profile.csv", usecols = att_select)
# profile['Year'] = 'Y' + profile['Year'].astype(str)
profile

,No.,Name,Prog,Year,Gender,Table
0,1,AARON YANG SEN WEI,MATH,Y3,M,1
1,13,LAM HUI QING,BUS,Y2,F,1
2,19,MUHAMMAD SOFYAN BIN ABDUL RAHIM,BUS,Y2,M,1
3,28,WANG WAYNE,BCG,Y3,M,1
4,24,TEE ZHENG YANG,BCG,Y4,M,2
5,8,DYLAN KOON MENG XIANG,BUS,Y2,M,2
6,14,LEK ZHI YING,BCG,Y3,F,2
7,21,ORE CHEE HUAI,BUS,Y2,M,2
8,18,MALCOLM TAN YEN DA,BUS,Y3,M,3
9,25,TENG WEI SHAO,BUS,Y2,M,3


### Gender

In [3]:
profile['Gender'].value_counts()

M    20
F    10
Name: Gender, dtype: int64

### Year

In [4]:
profile['Year'].value_counts()

Y3    14
Y2    11
Y4     5
Name: Year, dtype: int64

### Prog

In [5]:
profile['Prog'].value_counts()

BUS     14
BCG     12
MATH     4
Name: Prog, dtype: int64

### Setting number of groups

In [7]:
N_stu = len(profile['Gender'])
N_grp = 6

In [9]:
dm = pd.get_dummies(profile['Prog'])
df = pd.concat([profile, dm], axis=1)

dm = pd.get_dummies(profile['Year'])
df = pd.concat([df, dm], axis=1)

dm = pd.get_dummies(profile['Gender'])
data = pd.concat([df, dm], axis=1)
# data = data.rename(columns={2: "Y2", 3: "Y3", 4: "Y4"})
m = ro.Model('Group Partition')

x = m.dvar((N_stu, N_grp), 'B')

dis = m.dvar( N_stu, 'B' )

table = profile['Table'].values

m.min( dis.sum() )

### Total number of students to be assigned

In [10]:
m.st( x.sum() == N_stu ) 

### Each student can only be assigned to one group

In [11]:
m.st( x.sum(axis = 1) == 1 ) 

### Each group can not be more than 5 students

In [12]:
m.st( x.sum(axis = 0) <= np.ceil(N_stu/N_grp) ) 

### Students should be assigned to groups as evenly as possible based on their gender, year, and program.

In [13]:
criteria = ['Gender', 'Year', 'Prog']
for ct in criteria:
    ct_len = len(data[ct].value_counts())
    val_count = data[ct].value_counts()
    ct_val = val_count.index.values
    
    for i in range(ct_len):
        
        m.st( x.T @ data[ct_val[i]].to_numpy() >= np.floor( val_count[i]/N_grp ) ) 
        
        if np.floor( val_count[i]/N_grp ) == 0:
            m.st( x.T @ data[ct_val[i]].to_numpy() <= 1 )

### Most of the students will stay in the previous group

In [14]:
m.st( 3*dis[i] >= table[i] - sum( x[i,j]*j for j in range(N_grp)) for i in range(N_stu) )

m.st( table[i] >= sum( x[i,j]*j for j in range(N_grp)) for i in range(N_stu) )

In [15]:
# m.st(x >= 0)
# m.st(x <= 1)
    
m.solve(grb)

x_opt = x.get()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-14
Being solved by Gurobi...
Solution status: 2
Running time: 0.0148s


### Obtain the final list for each group

In [16]:
Group = {}
for i in range(N_grp):
    Group[i] = data[x_opt[:,i] == 1]['Name']
    print('Students in Group:', i + 1)
    print(Group[i])
    print('--------------------------')

Students in Group: 1
5     DYLAN KOON MENG XIANG
6              LEK ZHI YING
8        MALCOLM TAN YEN DA
9             TENG WEI SHAO
10              XU SHIGUANG
Name: Name, dtype: object
--------------------------
Students in Group: 2
0                  AARON YANG SEN WEI
1                        LAM HUI QING
2     MUHAMMAD SOFYAN BIN ABDUL RAHIM
3                          WANG WAYNE
12                 CARISSA LIM HUI YI
Name: Name, dtype: object
--------------------------
Students in Group: 3
4           TEE ZHENG YANG
7            ORE CHEE HUAI
11    SABRINA LAM JING WEN
13        JORDAN ANG YI DA
16      CHERLYN ANG KAI SI
Name: Name, dtype: object
--------------------------
Students in Group: 4
15            ARKAR MINN KHANT
17                   LI WENHAN
18          LOW SOO YEE CALVIN
19               BAIRI SAHITYA
21    CHENG XIAO YAN, CHEYENNE
Name: Name, dtype: object
--------------------------
Students in Group: 5
14              TOH CHIA LING
20             CAO TIEN THANH
23 

### Add to the profile and save it

In [ ]:
profile['Table'] = [np.where(x_opt[i] == 1)[0][0] + 1 for i in range(N_stu)]

In [ ]:
profile.to_csv('Sitting_Plan.csv')